In [2]:
%matplotlib inline
from skimage.io import imread, imshow
from skimage.transform import resize
from matplotlib.pyplot import figure, subplot, plot, yticks, xticks, legend
from math import ceil

def imshow_downscale(img):
    "matplotlib is slooooow, showing downscaled images is speedier"
    y,x = img.shape
    # downscale factor
    k = ceil(y//512+0.1)
    imshow(resize(img, (y//k, x//k)))

    # real ticks
    if y > x:
        # less ticks if not square image
        y_step = y//6
        x_step = int(y_step*x/y)
    if x < y:
        y_step = y//6
        x_step = int(y_step*y/x)
    else:
        y_step = y//6
        x_step = x//6

    yticks(range(0,y//k,y_step//k), [str(y) for y in range(0,y,y_step)])
    xticks(range(0,x//k,x_step//k), [str(x) for x in range(0,x,x_step)]);

def side_by_side(*images):
    "view images side by side"
    size = len(images)
    figure(figsize=(size*5,5))
    for i,img in enumerate(images):
        subplot(1, size, i+1)
        imshow_downscale(img)

In [3]:
import numpy as np

def stitch(img1, img2, dy, dx):
    y,x = img1.shape
    yy,xx = img2.shape
    
    if dy >= 0:
        y_pos1 = slice(0, y)
        y_pos2 = slice(dy, yy+dy)
    if dy < 0:
        y_pos1 = slice(-dy, y-dy)
        y_pos2 = slice(0, yy)
    if dx >= 0:
        x_pos1 = slice(0, x)
        x_pos2 = slice(dx, xx+dx)
    if dx < 0:
        x_pos1 = slice(-dx, x-dx)
        x_pos2 = slice(0, xx)
    
    Y = max((y_pos1.stop, y_pos2.stop)) - min((y_pos1.start, y_pos2.start))
    X = max((x_pos1.stop, x_pos2.stop)) - min((x_pos1.start, x_pos2.start))
    
    merged = np.zeros((Y,X), dtype=np.int)
    merged[y_pos1,x_pos1] += img1
    merged[y_pos2,x_pos2] += img2
    
    # average the overlap
    overlap = (slice(dy, y), slice(dx, x))
    merged[overlap] //= 2
    
    return merged.astype(np.uint8)